## Week 13 지리데이터 시각화(2)

### 가장 뜨는 제주도 핫플레이스는?

#### 데이터 준비: 인스타그램 크롤링 데이터
 - 제주도맛집으로 검색 후 Content, Date, Like, Place, Tags들을 추출

In [1]:
import pandas as pd
raw_total=pd.read_excel('1_crawling_raw.xlsx')
raw_total.head()

,content,date,like,place,tags
0,더 늦기전에 제주도로 떠나쟈😍.#제주핫플레이스 간단히 1탄 모아봄!#제주여행 요 필...,2018-12-07,"1,402",NaN,"['#제주핫플레이스', '#제주여행', '#제주여행', '#제주도여행', '#제주가..."
1,12월에 제주도를 가야하는이유🍊.#제주핫플 모음 끝판왕😍이거 하나면 겨울 #제주여행...,2018-12-03,368,NaN,"['#제주핫플', '#제주여행', '#제주', '#제주도', '#제주도맛집', '#..."
2,11월 놓치지 말아야 할 제주 관광.가을바람이 최고조에 이르는 11월추운 겨울을 앞...,2018-11-02,166,NaN,"['#honestin', '#어니스틴', '#제주여행', '#제주', '#제주도',..."
3,국민학교세대#제주관광#제주살이#제주이주민#아라동주민#삼남매집 #새해첫날#드라이브#명...,2019-01-01,28,명월국민학교,"['#제주관광', '#제주살이', '#제주이주민', '#아라동주민', '#삼남매집'..."
4,#제주관광 #제주 #돔나이트 #스트레스 #풀자 #춤추며 #땀날려 #가끔은괜찮아 #인...,2019-01-01,12,NaN,"['#제주관광', '#제주', '#돔나이트', '#스트레스', '#풀자', '#춤추..."


#### 위치정보 가져오기
 - value_counts() 함수를 이용하여 place의 컬럼의 값 가져오기

In [2]:
location_counts=raw_total['place'].value_counts()
location_counts

Jeju                    271
Jeju-do                 179
Jungle Book by Alice    108
Seogwipo                 66
제주도 크리스마스 박물관            59
                       ... 
제주마당                      1
형제섬                       1
동복해녀식당                    1
종로 광장시장                   1
디스이즈핫                     1
Name: place, Length: 1028, dtype: int64

 #### 등록된 위치정보별 빈도수 데이터 저장하기
 - 위의 데이터를 location_counts_df라는 dataframe에 저장하고, excel파일로 추출 

In [3]:
location_counts_df=pd.DataFrame(location_counts)
location_counts_df.head()

,place
Jeju,271
Jeju-do,179
Jungle Book by Alice,108
Seogwipo,66
제주도 크리스마스 박물관,59


In [4]:
location_counts_df.to_excel('2_location_counts.xlsx')

#### 위치정보에 어떠한 값들이 있는지 확인하기

In [5]:
locations=list(location_counts.index)
locations

['Jeju',
 'Jeju-do',
 'Jungle Book by Alice',
 'Seogwipo',
 '제주도 크리스마스 박물관',
 '할로비치',
 '제주에인감귤밭',
 'Jeju Island',
 '성산일출봉 城山日出峰  Seongsan Ilchulbong',
 '폼포코식당_pompokokitchen',
 '1100고지',
 'Nimome',
 '월정리해변',
 '고집돌우럭중문점',
 '석부작박물관',
 '제주도 애월읍',
 '알뜨르 비행장',
 '카멜리아 힐',
 '제주레포츠랜드',
 '밥깡패',
 '제주 함덕 서우봉 해변',
 '제주도 서귀포 중문관광단지 濟州島西歸浦中文觀光團地',
 '제주고궁한복카페 jeju gogung hanbok studio',
 '위미동백나무군락지',
 '제주신화월드 Jeju Shinhwa World',
 'Woljeongri Beach',
 '협재해변 Beach',
 '연동 바오젠 거리',
 '김녕미로공원 Jeju Kimnyoung Maze Park',
 '고래배꼽',
 '제주 성산 신산포구자연산횟집 회포장',
 '새별오름',
 '용머리해안',
 '용눈이오름',
 '캔디원',
 '사려니숲길',
 '하이엔드 제주',
 '제주빅볼랜드',
 '서귀포 느영나영 게스트하우스',
 '휴애리 자연생활공원',
 '옹포리',
 '삼무공원',
 '마마뜰',
 '제주어린왕자게스트하우스',
 '위미동백나무군락',
 '제주커피박물관 Baum',
 'Jeju Aewol',
 '한라산',
 '위미2리 동백군락지',
 'Jeju Island 제주특별자치도 济州道',
 '광치기해변',
 '서우봉',
 '제주해남',
 '한라산 (漢拏山, Hallasan)',
 '카페한라산',
 '성이시돌목장',
 '제주 송악산',
 '옹포별장가든',
 '월정리카페콧수염',
 '牛岛 Udo Island 우도',
 '곽지해수욕장',
 '카페브리프',
 '아줄레주',
 '에코랜드',
 '산굼부리',
 '제주 꿈꾸는고래 스쿠버&게스트하우스',
 'Terarosa - 테라로사',

#### 카카오 검색 API 사용하기

In [28]:
import requests

searching = '합정 스타벅스'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)
headers = {
    "Authorization": "KakaoAK 96d5d24579c5129d2e3c66e9f18a07e4"
}
places = requests.get(url, headers = headers).json()['documents']
places

[{'address_name': '서울 마포구 합정동 472',
  'category_group_code': 'CE7',
  'category_group_name': '카페',
  'category_name': '음식점 > 카페 > 커피전문점 > 스타벅스',
  'distance': '',
  'id': '2057327896',
  'phone': '1522-3232',
  'place_name': '스타벅스 합정점',
  'place_url': 'http://place.map.kakao.com/2057327896',
  'road_address_name': '서울 마포구 월드컵로1길 14',
  'x': '126.91253700818196',
  'y': '37.54994959743763'},
 {'address_name': '서울 마포구 합정동 473',
  'category_group_code': 'CE7',
  'category_group_name': '카페',
  'category_name': '음식점 > 카페 > 커피전문점 > 스타벅스',
  'distance': '',
  'id': '2132676511',
  'phone': '1522-3232',
  'place_name': '스타벅스 합정푸르지오점',
  'place_url': 'http://place.map.kakao.com/2132676511',
  'road_address_name': '서울 마포구 월드컵로3길 14',
  'x': '126.91247704039314',
  'y': '37.550909116022105'},
 {'address_name': '서울 마포구 서교동 490',
  'category_group_code': 'CE7',
  'category_group_name': '카페',
  'category_name': '음식점 > 카페 > 커피전문점 > 스타벅스',
  'distance': '',
  'id': '288597324',
  'phone': '1522-3232',

#### 카카오 검색 API를 활용한 장소 검색
- 안내 페이지(카카오 키워드 장소검색 API 사용 안내): http://bit.ly/pwd_kakaoAPI_guide

In [29]:
def find_places(searching):
    url='https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)
    headers={"Authorization":"KakaoAK 96d5d24579c5129d2e3c66e9f18a07e4"}
    places=requests.get(url,headers=headers).json()['documents']
    place=places[0]
    name=place['place_name']
    x=place['x']
    y=place['y']
    data=[name,x,y,searching]
    return data

In [32]:
data=find_places('제주공항')
data

['제주국제공항', '126.492769004244', '33.5070789578184', '제주공항']

#### 카카오 API를 활용하여 인스타그램 위치명 위치정보 검색하기

In [37]:
from tqdm.notebook import tqdm

In [38]:
import time
locations_inform=[]

for location in tqdm(locations):
    try:
        data=find_places(location)
        locations_inform.append(data)
        time.sleep(0.5)
    except:
        pass
locations_inform

  0%|          | 0/1028 [00:00<?, ?it/s]

[['CENACULUM JEJU', '126.30129764229179', '33.4449549374343', 'Jeju'],
 ['서귀포잠수함', '126.558616052674', '33.2393033784206', 'Seogwipo'],
 ['바이나흐튼 크리스마스박물관', '126.32785786016811', '33.2915720831759', '제주도 크리스마스 박물관'],
 ['할로비치 본점', '126.2815248933638', '33.30832616522123', '할로비치'],
 ['제주에인감귤밭', '126.5390139270361', '33.25656069063887', '제주에인감귤밭'],
 ['제주도', '126.54587355630036', '33.379777816446165', 'Jeju Island'],
 ['한라산1100고지', '126.462219691112', '33.3580781709788', '1100고지'],
 ['월정리해수욕장', '126.795805057888', '33.556469394054', '월정리해변'],
 ['고집돌우럭 중문점', '126.416704762779', '33.2579811121134', '고집돌우럭중문점'],
 ['숨도', '126.53652102219156', '33.25045551460648', '석부작박물관'],
 ['올레길 16코스(고내-광령 올레)', '126.38979561557086', '33.46649759191228', '제주도 애월읍'],
 ['알뜨르비행장', '126.271527320164', '33.2047161778831', '알뜨르 비행장'],
 ['카멜리아힐', '126.370009467525', '33.2896344311608', '카멜리아 힐'],
 ['제주레포츠랜드', '126.6387551242765', '33.48206819907232', '제주레포츠랜드'],
 ['밥깡패', '126.26342309622974', '33.41011846869669', '밥

#### 위치정보 저장하기 및 위치 데이터 병합하기

In [42]:
locations_inform_df = pd.DataFrame(locations_inform)
locations_inform_df.columns = ['name_official','경도','위도','인스타위치명']
locations_inform_df.to_excel('3_locations.xlsx', index=False)

In [43]:
location_counts_df = pd.read_excel('2_location_counts.xlsx', index_col = 0)
locations_inform_df = pd.read_excel('3_locations.xlsx')

In [45]:
location_data = pd.merge(locations_inform_df, location_counts_df,
how = 'inner', left_on = 'name_official', right_index=True)
location_data.head()

,name_official,경도,위도,인스타위치명,place
1,서귀포잠수함,126.558616,33.239303,Seogwipo,1
614,서귀포잠수함,126.558616,33.239303,서귀포잠수함,1
4,제주에인감귤밭,126.539014,33.256561,제주에인감귤밭,48
5,제주도,126.545874,33.379778,Jeju Island,4
84,제주도,126.545874,33.379778,제주도,4


#### 데이터 중복 점검하기

In [46]:
location_data['name_official'].value_counts()

제주동문재래시장    4
오설록티뮤지엄     3
자매국수 본점     3
서귀포잠수함      2
칠돈가 본점      2
           ..
이게밥도둑       1
숨비아일랜드      1
삼공주회센터      1
토끼썸         1
디스이즈핫       1
Name: name_official, Length: 363, dtype: int64

#### 장소 이름 기준 병합하기 (agg 함수를 이용하여 name_official','경도','위도'가 동일하면 place 값을 병합)

In [50]:
location_data = location_data.pivot_table(index = ['name_official','경도','위도'], values=location_data.head())

In [51]:
location_data.to_excel('4_location_inform.xlsx')

#### folium을 이용한 지도 시각화 ① - 개별 표시

In [52]:
location_data = pd.read_excel('4_location_inform.xlsx')
location_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364 entries, 0 to 363
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name_official  363 non-null    object 
 1   경도             364 non-null    float64
 2   위도             364 non-null    float64
 3   place          364 non-null    int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 11.5+ KB


In [57]:
import folium
Mt_Hanla =[33.362500, 126.533694]
map_jeju = folium.Map(location = Mt_Hanla, zoom_start = 11)
for i in range(len(location_data)):
    name = location_data ['name_official'][i] 
    count = location_data ['place'][i]
    size = int(count)*2
    long = float(location_data['위도'][i])
    lat = float(location_data['경도'][i])
    folium.CircleMarker((long,lat), radius = size, color='red', popup=name).add_to(map_jeju)
map_jeju

In [58]:
map_jeju.save('jeju.html')

In [60]:
from folium.plugins import MarkerCluster
locations = []
names = []
for i in range(len(location_data)):
    data = location_data.iloc[i]
    locations.append((float(data['위도']),float(data['경도'])))
    names.append(data['name_official'])
    Mt_Hanla =[33.362500, 126.533694]
    map_jeju2 = folium.Map(location = Mt_Hanla, zoom_start = 11)
    marker_cluster = MarkerCluster(locations=locations, popups=names, name='Jeju', overlay=True, control=True,)
marker_cluster.add_to(map_jeju2)
folium.LayerControl().add_to(map_jeju2)
map_jeju2

In [61]:
map_jeju2.save('jeju_cluster.html')